<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/finaltask/LatstTask_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!git clone https://github.com/saiku122/AIJobcolle.git

fatal: destination path 'AIJobcolle' already exists and is not an empty directory.


In [19]:
cd /content/AIJobcolle/MachineLearning/finaltask/

/content/AIJobcolle/MachineLearning/finaltask


★★最終課題★★</br>
〇入力データ</br>
・モデル用：final_hr_analysis_train.csv</br>
・スコア用：final_hr_analysis_test.csv</br>
</br>
〇出力データ</br>
・スコア付データ：</br> 
・モデル：</br>


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [21]:
#インプットファイル
file_model_name = 'final_hr_analysis_train'
file_test_name = 'final_hr_analysis_test'

In [22]:
my_dtype = {'sales':object,
            'salary':object} ##!!!変更箇所
ohe_cols = ['sales',
           'salary'] #!!!変更箇所

In [23]:
#  モデル用データ読み込み
df = pd.read_csv('./input/'+ file_model_name + '.csv',
                 header=0,
                 dtype=my_dtype)
ID = df.iloc[:,0] #1列目取得
y=df.iloc[:,1]    #2列目の正解データを取得
X=df.iloc[:,2:]   #3列目以降取得
df.dtypes

index                      int64
left                       int64
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [24]:
# one-hot encoding処理
X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe = X_ohe.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns = X_ohe.columns.values #モデルデータの列確定

In [25]:
#欠損値の置き換え
imp=SimpleImputer() #平均値で欠損値を補完
imp.fit(X_ohe)
X_ohe = pd.DataFrame(imp.transform(X_ohe),columns=X_ohe_columns) # モデルデータの列を平均値補完適用させ変数を更新

In [26]:
#特徴量抽出
selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05) #特徴量をRFE(主成分分析)を用いて次元削減する。(モデルはランダムフォレスト)
selector.fit(X_ohe, y) #RFEした結果を保存
X_ohe_selected = selector.transform(X_ohe) #X_oheに反映
X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_]) #削減実行後のデータを再構成

**テストデータの読み込み**<br>
モデル用データのエンコード結果に列名をあわせる

In [27]:
#  テスト用データ読み込み
df_s = pd.read_csv('./input/'+ file_test_name + '.csv',
                 header=0,
                 dtype=my_dtype)
ID_s = df_s.iloc[:,[0]] #1列目取得
y_s=df_s.iloc[:,1]    #2列目の正解データを取得
X_s=df_s.iloc[:,2:]   #3列目以降取得

In [28]:
# one-hot encoding処理
X_ohe_s = pd.get_dummies(X_s, dummy_na=True, columns=ohe_cols)
#display(X_ohe)
X_ohe_s = X_ohe_s.dropna(axis=1,how='all')#axis=1　列
#display(X_ohe)
X_ohe_columns_s = X_ohe_s.columns.values #テストデータの仮確定

In [29]:
#列の整合性チェック
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: set()
スコアのみに存在する項目: set()


In [30]:
#列名一致
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
#display(X_ohe_s2.head(3))

set_Xm = set(X_ohe.columns.values)
set_Xs = set(X_ohe_s.columns.values)

#スコアにのみ存在する項目の削除
X_ohe_s3 = X_ohe_s2.drop(list(set_Xs-set_Xm),axis=1)

#print(X_ohe_s3.shape)
#display(X_ohe_s3.head(3))

#list(set_Xs-set_Xm)

In [31]:
#スコアにのみ存在する項目の削除
X_ohe_s3.loc[:,list(set_Xm-set_Xs)] = X_ohe_s3.loc[:,list(set_Xm-set_Xs)].fillna(0,axis=1)
X_ohe_s3.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.44,0.57,2.0,141.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.55,0.96,3.0,194.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.72,0.67,5.0,210.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [32]:
#ソート順の統一
X_ohe_s3 = X_ohe_s3.reindex(X_ohe.columns.values,axis=1)
X_ohe_s3.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,sales_nan,salary_high,salary_low,salary_medium,salary_nan
0,0.44,0.57,2.0,141.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.55,0.96,3.0,194.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.72,0.67,5.0,210.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
#欠損値処理
X_ohe_s4 = pd.DataFrame(imp.transform(X_ohe_s3),columns=X_ohe_columns)

In [34]:
#RFE適用後の列名のみテストデータから抽出
X_ohe_selected_s=X_ohe_s4.loc[:, X_ohe_columns[selector.support_]]

In [35]:
#列の整合性再確認チェック ←ログとしてははきだすべき
cols_model = set(X_ohe_selected.columns.values)
cols_score = set(X_ohe_selected_s.columns.values)
diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

モデルのみに存在する項目: set()
スコアのみに存在する項目: set()


In [36]:
X_ohe_selected_s

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company
0,0.44,0.57,2.0,141.0,3.0
1,0.55,0.96,3.0,194.0,3.0
2,0.72,0.67,5.0,210.0,2.0
3,0.96,0.75,4.0,177.0,2.0
4,0.96,0.54,3.0,198.0,3.0
...,...,...,...,...,...
4495,0.56,0.71,3.0,211.0,6.0
4496,0.58,0.79,5.0,262.0,2.0
4497,0.85,0.58,4.0,273.0,4.0
4498,0.54,0.64,6.0,278.0,2.0


**モデリング**

In [37]:
#アルゴリズム定義
#model_name = 'GBC_001'
'''
clf = Pipeline([('scl',StandardScaler()), 
                ('est',GradientBoostingClassifier(random_state=1))
                ])
'''
# set pipelines for different algorithms
pipelines = {
    'knn':
       Pipeline([('scl',StandardScaler()),
                  ('est',KNeighborsClassifier())]),
    'logistic':
        Pipeline([('scl',StandardScaler()),
                  ('est',LogisticRegression(random_state=1))]),
    'rsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1,probability = True))]),
    'lsvc':
        Pipeline([('scl',StandardScaler()),
                  ('est',SVC(C=1.0, kernel='linear', class_weight='balanced', random_state=1,probability = True))]),
    'tree':
        Pipeline([('scl',StandardScaler()),
                  ('est',DecisionTreeClassifier(random_state=1))]),
    'rf':
        Pipeline([('scl',StandardScaler()),
                  ('est',RandomForestClassifier(random_state=1))]),
    'gb':
        Pipeline([('scl',StandardScaler()),
                  ('est',GradientBoostingClassifier(random_state=1))]),
    'lgb':
        Pipeline([('scl',StandardScaler()),
                  ('lgb',lgb.LGBMClassifier())]),
    'mlp': #ニューラルネットワーク
        Pipeline([('scl',StandardScaler()),
                  ('est',MLPClassifier(hidden_layer_sizes=(3,3),
                                       max_iter=1000,
                                       random_state=1))])
}

In [43]:
#clf.fit(X_ohe_selected, y.values.ravel())#Xを標準化、勾配ブースティングを適用)
scores = {}
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_ohe_selected, y.values.ravel())
#    scores[(pipe_name,'train')] = accuracy_score(y, pipeline.predict(X_ohe_selected))
#    scores[(pipe_name,'test')] = accuracy_score(y_s, pipeline.predict(X_ohe_selected_s))
    print(pipe_name)
    print("========================")
    print(pipeline.predict_proba(X_ohe_selected_s))
    print("========================")
#pd.Series(scores).unstack()

knn
[[0.  1. ]
 [1.  0. ]
 [1.  0. ]
 ...
 [1.  0. ]
 [0.8 0.2]
 [1.  0. ]]
logistic
[[0.65508039 0.34491961]
 [0.7063478  0.2936522 ]
 [0.92408535 0.07591465]
 ...
 [0.89164479 0.10835521]
 [0.85359663 0.14640337]
 [0.36536663 0.63463337]]
rsvc
[[0.07161185 0.92838815]
 [0.9875506  0.0124494 ]
 [0.96980056 0.03019944]
 ...
 [0.97591312 0.02408688]
 [0.86640174 0.13359826]
 [0.99025139 0.00974861]]
lsvc
[[0.6819394  0.3180606 ]
 [0.74834405 0.25165595]
 [0.92630803 0.07369197]
 ...
 [0.8340895  0.1659105 ]
 [0.91271505 0.08728495]
 [0.43666082 0.56333918]]
tree
[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
rf
[[0.   1.  ]
 [1.   0.  ]
 [1.   0.  ]
 ...
 [0.98 0.02]
 [0.8  0.2 ]
 [1.   0.  ]]
gb
[[0.04250815 0.95749185]
 [0.9861997  0.0138003 ]
 [0.98966501 0.01033499]
 ...
 [0.95288159 0.04711841]
 [0.97848763 0.02151237]
 [0.97156627 0.02843373]]
lgb
[[0.008246   0.991754  ]
 [0.99815784 0.00184216]
 [0.99630932 0.00369068]
 ...
 [0.95979255 0.04020745]
 [0.905724   0.09

In [45]:
for pipe_name, pipeline in pipelines.items():
  #results = cross_val_score(pipeline, X_ohe_selected, y, scoring='roc_auc', cv=5)
  #print('cv score('+pipe_name+'):', np.average(results), '+-', np.std(results))
  print(pipe_name)
  predscore = pd.DataFrame((pipeline.predict_proba(X_ohe_selected_s)[:,1:2]), columns=['pred_score'])
  ID_s.join(predscore).to_csv('./output/'+  pipe_name + '_' + file_test_name + '_with_pred.csv', index=False)

  #imp = pd.DataFrame([pipeline.named_steps['est'].feature_importances_], columns=X_ohe_columns[selector.support_])
  #imp.T.to_csv('./output/'+  pipe_name + '_feature_importances.csv', index=True)

knn
logistic
rsvc
lsvc
tree
rf
gb
lgb
mlp


In [46]:
#モデル保存
for pipe_name, pipeline in pipelines.items():
  filename=pipe_name + '.joblib'
  joblib.dump('./output/'+ pipe_name, filename)